## **Maestría en Inteligencia Artificial Aplicada**
### **Curso: Inteligencia Artificial y Aprendizaje Automático**
#### Tecnológico de Monterrey
####P rof Luis Eduardo Falcón Morales

### **Adtividad de la Semana 8**
#### **Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Esta Actividad deberás resolverla de manera individual.**

**Nombre y matrícula:**

*   
*   


In [11]:
# Incluye aquí todas las librerías y paquetes que requieras.
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD








### **Liga de Datos de la UCI "Restaurant & consumer data".**
### **Descarga "RCdata.zip" y de ahí necesitaremos los archivos "rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data



In [2]:
# Descarga los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data1 = pd.read_csv("rating_final.csv", header='infer', sep=",")
data2 = pd.read_csv("geoplaces2.csv", header='infer',  encoding='latin-1')

print(data1.shape, data2.shape)


(1161, 5) (130, 21)


## **Ejercicio - 1**

### **Explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.**

El argumento **latin-1**se utiliza para especificar la codificación de caracteres con la que se debe leer el archivo. Esto es necesario cuando el archivo contiene caracteres que no están presentes en la codificación por defecto (usualmente UTF-8 en muchas librerías de Python). **Latin-1** abarca los caracteres usados en las lenguas de Europa Occidental, lo cual es apropiado para datos que pueden incluir nombres de lugares, direcciones o descripciones en español, como es el caso del dataset geoplaces2.csv, el cual incluye nombres y direcciones de restaurantes en México.
Usar esta codificación asegura que los caracteres especiales, como acentos y ñ, se lean correctamente.

In [3]:
# Del primer archivo obtenemos una matriz con 3 evaluaciones de los restaurantes:
# general, comida y servicio.
# Las evaluaciones pueden ser 0, 1 o 2, siendo 0 la menor calificación y 2 la
# mayor calificación:

data1.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [4]:
# Del segundo archivo obtenemos información diversa de cada restaurante:

data2.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [5]:
# De data1 no requeriremos "rating" y de data2 solo necesitamos "placeID" y "name":
# Definimos la lista y matrices con los factores que necesitamos:

lista_data1 = ['userID','placeID','food_rating','service_rating']
lista_data2 = ['placeID','name']

data1a = data1[lista_data1]
data2a = data2[lista_data2]

## **Ejercicio - 2**

In [6]:
# Define el DataFrame que conjunte la información de las dos DataFrame data1a
# y data2a en uno solo, a través de la columna común "placeID" y que
# llamaremos "df_combinado":


# ************* Inlcuye aquí tu código:*****************************


df_combinado = pd.merge(data1a, data2a, on='placeID')


# *********** Aquí termina la sección de agregar código *************


# Despleguemos la dimensión y los primeros renglones de este DataFrame:

print(df_combinado.shape)
df_combinado.head()

(1161, 5)


,userID,placeID,food_rating,service_rating,name
0,U1077,135085,2,2,Tortas Locas Hipocampo
1,U1108,135085,2,1,Tortas Locas Hipocampo
2,U1081,135085,2,1,Tortas Locas Hipocampo
3,U1056,135085,2,2,Tortas Locas Hipocampo
4,U1134,135085,1,2,Tortas Locas Hipocampo


## **Ejercicio - 3**

In [7]:
#    Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación de la comida (food_rating). La llamaremos "UtMx_food".


# ************* Inlcuye aquí tu código:*****************************


UtMx_food = df_combinado.pivot_table(index='name', columns='userID', values='food_rating', fill_value=0)


# *********** Aquí termina la sección de agregar código *************


print('Dimensión de la matriz de Utilidad sobre la evaluación de la comida:')
print('(restaurantes, usuarios) =', (UtMx_food.shape))
UtMx_food.head()

Dimensión de la matriz de Utilidad sobre la evaluación de la comida:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,2,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## **Ejercicio - 4**

### **En la Factorización SVD la cantidad de valores singulares será menor o igual al menor valor de los renglones o columnas de la matriz de utilidad UtMx.**

### **La factorización SVD de una matriz $A$ tiene la forma:**

$A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

In [10]:
# Apliquemos la transformación SVD para obtener la matriz de las variables
# latentes de los restaurantes en relación a la evalución de los usuarios con
# respecto a la comida. Deberás obtener la factorización SVD con el máximo de
# comonentes y por lo tanto el máximo de valores singulares posible.
# Asigna a la variable "nc_food" el valor del máximo número de valores singulares
# que podemos obtener de la utilidad que generamos en el ejercicio anterior.


# ************* Inlcuye aquí tu código:*****************************


nc_food = 20     # número de componentes


# *********** Aquí termina la sección de agregar código *************


# Inicializamos y aplicamos la factorización SVD en relación a la
# evaluación de la comida, para determinar la cantidad de componentes
# que podemos utilizar con un 90% de explicación de la variabilidad
# de dichas componentes:

SVD_food = TruncatedSVD( n_components = nc_food )
SVD_food.fit(UtMx_food)

# Determinemos la cantidad de componentes que explican un 90%
# de la variabilidad acumulada de cada componente:

for j in range(nc_food):
  if SVD_food.explained_variance_ratio_[0:j].sum() > 0.90:
    break

# Usaremos esta cantidad de componentes para las recomendaciones basadas
# en la calificación de la comida:
N_food = j-1

print('Total de valores singulares basados en evaluación de la :', nc_food)
print('Valor de truncamiento al 90% de dicha variabilidad:',N_food)


Total de valores singulares basados en evaluación de la : 20
Valor de truncamiento al 90% de dicha variabilidad: 18


In [12]:
# Usando la métrica de correlación de Pearson, obtendremos las 10 mejores
# recomendaciones de este modelo no supervisado con base a la información de
# alguien que desea obtener las similidades com el restaurante "Restaurante Pueblo
# Bonito" y con la cota de truncamiento obtenido en el ejercicio anterior.
# Para ello obtendremos las correlaciones positivas y de ahí deberás seleccionar
# las diez mejores.

# Factorización SVD:
SVD_food = TruncatedSVD(n_components = N_food)
resultant_matrix_food = SVD_food.fit_transform(UtMx_food)

# Matriz de correlación de Pearson:
corr_mat_food = np.corrcoef(resultant_matrix_food)

# Restaurante de referencia:
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_food.T.columns  # nombres de restaurantes
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_food[idx_rest] # Vector de Correlación del RinconHuasteco

# Buscando las correlaciones positivas:
idx = (corr_rest>0)
mejores_sim_food = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_food.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_food))

print('Algunos de los resultados encontrados:')
mejores_sim_food[0:7]

Total de similaridades positivas encontradas: 53
Algunos de los resultados encontrados:


[(0.0659063826200899, 'Abondance Restaurante Bar'),
 (0.2701710226462708, 'Cafeteria y Restaurant El Pacifico'),
 (0.7165006328820472, 'El Club'),
 (0.1997085090508264, 'El Herradero Restaurante and Bar'),
 (0.0688341286458509, 'El Mundo de la Pasta'),
 (0.047594094674298607, 'El Rincón de San Francisco'),
 (0.0868861560452667, 'El angel Restaurante')]

## **Ejercicio - 5**

In [14]:

corr_mat_food_corrected = np.corrcoef(resultant_matrix_food)  # Correlación entre restaurantes basada en las filas

corr_rest_corrected = corr_mat_food_corrected[idx_rest]  # Correlación correcta del restaurante de referencia
idx_corrected = (corr_rest_corrected > 0)
mejores_sim_food_corrected = [(corr_rest_corrected[i], nombres_rest[i]) for i in range(len(nombres_rest)) if idx_corrected[i]]

# Ordenar la lista de recomendaciones de manera descendente y correctamente
mejores_sim_food_ordenadas_corrected = sorted(mejores_sim_food_corrected, key=lambda x: x[0], reverse=True)

print('Total de similaridades positivas encontradas:', len(mejores_sim_food_ordenadas_corrected))
print('Algunos de los resultados encontrados:')
mejores_sim_food_ordenadas_corrected[:7]  # Mostrando los primeros 7 resultados corregidos


Total de similaridades positivas encontradas: 53
Algunos de los resultados encontrados:


[(1.0, 'Restaurante Pueblo Bonito'),
 (0.9162313363578395, 'pizza clasica'),
 (0.9023403293400675, 'Restaurante la Estrella de Dima'),
 (0.8095640283064203, 'Restaurante Guerra'),
 (0.7165006328820472, 'El Club'),
 (0.5540391672332174, 'Pizzeria Julios'),
 (0.45695576376494235, 'Hamburguesas saul')]

## **Ejercicio - 6**

### **De manera análoga y usando ahora la evaluación con respecto al servicio (service_rating), encuentrarás ahora los diez restaurantes con mayor similaridad al "Restaurante Pueblo Bonito" y con respecto a la métrica de similaridad de Pearson.**

### **Deberás usar la cantidad de componentes necesarias para que la varabilidad de la factorización SVD quede explicada en un 90%.**

### **a.  Para ello define primero la matriz de utilidad, UtMx_service, donde los renglones son los nombres de los restaurantes, las columnas los usuarios y las entradas las evaluaciones con respecto al servicio recibido en el restaurante.**





In [16]:

# Definir la matriz de utilidad para evaluaciones del servicio
UtMx_service = df_combinado.pivot_table(index='name', columns='userID', values='service_rating', fill_value=0)

# Determinar el número de componentes para SVD que expliquen al menos el 90% de la variabilidad
SVD_service_trial = TruncatedSVD(n_components=20)
SVD_service_trial.fit(UtMx_service)
varianza_acumulada = SVD_service_trial.explained_variance_ratio_.cumsum()

nc_service = 20  # Empezar con un número bajo de componentes
while varianza_acumulada[-1] < 0.9 and nc_service < min(UtMx_service.shape):
    nc_service += 10
    SVD_service_trial = TruncatedSVD(n_components=nc_service)
    SVD_service_trial.fit(UtMx_service)
    varianza_acumulada = SVD_service_trial.explained_variance_ratio_.cumsum()

# Aplicar SVD con el número óptimo de componentes
N_service = nc_service
SVD_service = TruncatedSVD(n_components=N_service)
resultant_matrix_service = SVD_service.fit_transform(UtMx_service)

# Calcular la matriz de correlación de Pearson para las variables latentes
corr_mat_service = np.corrcoef(resultant_matrix_service)

# Buscar las correlaciones positivas para el restaurante de referencia
restaurante_de_referencia = "Restaurante Pueblo Bonito"
idx_rest_service = list(UtMx_service.index).index(restaurante_de_referencia)
corr_rest_service = corr_mat_service[idx_rest_service]

# Seleccionar las correlaciones positivas y ordenarlas
mejores_sim_service = [(corr_rest_service[i], UtMx_service.index[i]) for i in range(len(UtMx_service.index)) if corr_rest_service[i] > 0]
mejores_sim_service_ordenadas = sorted(mejores_sim_service, key=lambda x: x[0], reverse=True)

# Mostrar los diez restaurantes con mayor similaridad
top_10_sim_service = mejores_sim_service_ordenadas[:10]
for sim, name in top_10_sim_service:
    print(f"{name}: {sim}")


Restaurante Pueblo Bonito: 1.0
pizza clasica: 0.6560913745938618
El Club: 0.5384824867728837
Hamburguesas saul: 0.49632185064602985
Restaurante la Estrella de Dima: 0.45862054764979143
Restaurante Guerra: 0.4161703157551229
Restaurant Orizatlan: 0.3762631214395379
Restaurante El Cielo Potosino: 0.370131993714761
la Cochinita Pibil Restaurante Yucateco: 0.3476137682361555
La Fontana Pizza Restaurante and Cafe: 0.313056582973095


In [17]:
print('Dimensión de la matriz de Utilidad sobre la evaluación del servicio:')
print('(restaurantes, usuarios) =', (UtMx_service.shape))
UtMx_service.head()

Dimensión de la matriz de Utilidad sobre la evaluación del servicio:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# b. Incluye el código necesario para obtener y desplegar los 10 restaurantes con
#    mayor similaridad de Pearson al "Restaurante Pueblo Bonito", y usando solo
#    el 90% de las componentes que explican la varianza de la matriz UtMx_service.
#    Desplegarlos en orden descendente, sin incliur el restaurante de referencia.

# ************* Inlcuye aquí tu código:*****************************
SVD_service_90 = TruncatedSVD(n_components=N_service+1)  # N_service ya calculado para explicar el 90%
resultant_matrix_service_90 = SVD_service_90.fit_transform(UtMx_service)

# Recalcular la matriz de correlación de Pearson con este número ajustado de componentes
corr_mat_service_90 = np.corrcoef(resultant_matrix_service_90)

# Obtener nuevamente el vector de correlación para el restaurante de referencia
corr_rest_service_90 = corr_mat_service_90[idx_rest_service]

# Filtrar para obtener solo las correlaciones positivas excluyendo el restaurante de referencia
mejores_sim_service_90 = [(corr_rest_service_90[i], UtMx_service.index[i]) for i in range(len(UtMx_service.index)) if corr_rest_service_90[i] > 0
                          and UtMx_service.index[i] != restaurante_de_referencia]

# Ordenar las correlaciones de mayor a menor, excluyendo el restaurante de referencia
mejores_sim_service_ordenadas_90 = sorted(mejores_sim_service_90, key=lambda x: x[0],
                                          reverse=True)

# Seleccionar y mostrar los top 10 restaurantes más similares
top_10_sim_service_90 = mejores_sim_service_ordenadas_90[:10]

# Desplegar los 10 restaurantes con mayor similaridad, en orden descendente, sin incluir el restaurante de referencia
top_10_sim_service_90




[(0.6551097396333586, 'pizza clasica'),
 (0.5379958883312017, 'El Club'),
 (0.4932545516904137, 'Hamburguesas saul'),
 (0.4595697607553897, 'Restaurante la Estrella de Dima'),
 (0.4154397287284569, 'Restaurante Guerra'),
 (0.3762775321720009, 'Restaurant Orizatlan'),
 (0.36992015259340555, 'Restaurante El Cielo Potosino'),
 (0.3473901147107743, 'la Cochinita Pibil Restaurante Yucateco'),
 (0.3123225422497308, 'La Fontana Pizza Restaurante and Cafe'),
 (0.31186680352597157, 'Restaurant Oriental Express')]

## **Ejercicio - 7**

### **Incluye tus comentarios y conclusiones de la actividad. En particular indica cuáles restaurantes encontraste con correlaciones altas tanto para la calificación de la comida, como del servicio.**

La lista de los diez restaurantes con sus coeficientes de correlación es la siguiente:

### 1.Pizza clásica: 0.6551
### 2.El Club: 0.5379
### 3.Hamburguesas saul: 0.4932
### 4.Restaurante la Estrella de Dima: 0.4595
### 5.Restaurante Guerra: 0.4154
### 6.Restaurant Orizatlan: 0.3762
### 7.Restaurante El Cielo Potosino: 0.3699
### 8.La Cochinita Pibil Restaurante Yucateco: 0.3473
### 10.La Fontana Pizza Restaurante and Cafe: 0.3123
### 11. Restaurant Oriental Express: 0.3118

# **Comentarios y Conclusiones**

**Coeficientes de Correlación Positivos**: Todos los restaurantes listados tienen una correlación positiva con el **"Restaurante Pueblo Bonito"**, indicando que los usuarios tienden a calificar el servicio de estos restaurantes de manera similar.

**Comparación con Calificaciones de Comida**: Si comparamos esta lista con la lista previa basada en las calificaciones de la comida, podemos identificar restaurantes que consistentemente tienen altas correlaciones en ambos servicios y comida. Esto sugiere que estos restaurantes son apreciados de manera **similar** por los consumidores, lo cual podría señalar una consistencia en la calidad general del restaurante.

**Recomendaciones Basadas en el Perfil del Consumidor**: Restaurantes con altas correlaciones tanto en comida como en servicio pueden ser especialmente recomendables para los consumidores que valoran una experiencia gastronómica equilibrada.

**Diversidad en la Oferta**: La lista incluye una variedad de restaurantes, desde pizzerías hasta establecimientos de comida regional, lo que sugiere que la similaridad en la percepción del servicio no está limitada a un tipo particular de cocina o estilo de restaurante.

**Consideraciones para el Sistema de Recomendación**: Para los sistemas de recomendación, considerar múltiples dimensiones de la experiencia en restaurantes, como comida y servicio, puede proporcionar una guía más completa para los usuarios.

>> **Fin de la Actividad de Sistemas de Recomendación**